In [1]:
from keras.preprocessing.image import ImageDataGenerator
from keras.applications.vgg19 import VGG19
from keras.layers import Flatten, Dense, Dropout, Conv2D, MaxPool2D,BatchNormalization, LSTM, CuDNNLSTM, MaxPooling2D
from keras.models import Sequential
from keras import regularizers
import keras

from keras.layers import TimeDistributed
from keras.layers import GlobalAveragePooling2D
from tensorflow.keras.applications import MobileNetV2

from tensorflow.keras.optimizers import Adam
import tensorflow as tf

import matplotlib.pyplot as plt
import cv2

In [2]:
image_generator = ImageDataGenerator(
    rescale=1./255, 
    horizontal_flip=2, 
    vertical_flip=2
)    

training_set = image_generator.flow_from_directory(batch_size=256,
                                                 directory="E:\\Final_year Project\\Training Dataset",
                                                 shuffle=True,
                                                 target_size=(48, 48), 
                                                 class_mode='categorical',
#                                                  color_mode = "grayscale"
                                                )

testing_set = image_generator.flow_from_directory(batch_size=128,
                                                 directory="E:\\Final_year Project\\Testing Dataset",
                                                 shuffle=True,
                                                 target_size=(48, 48), 
                                                 class_mode='categorical',
#                                                  color_mode = "grayscale"
                                                 )

Found 18670 images belonging to 7 classes.
Found 11200 images belonging to 7 classes.


In [3]:
training_set.class_indices

{'ANGRY': 0,
 'DISGUST': 1,
 'FEAR': 2,
 'HAPPY': 3,
 'NEUTRAL': 4,
 'SAD': 5,
 'SURPRISED': 6}

In [4]:
VGG_model = VGG19(
      input_shape = (48, 48, 3),
      include_top = False,
      weights = 'imagenet'
    )

In [6]:
for layers in VGG_model.layers:
    layers.trainable = False

In [7]:
x = Flatten()(VGG_model.output)
x = Dense(512, activation = "relu")(x)
x = Dropout(0.5)(x)
x = Dense(14, activation = "relu")(x)
x = Dense(7, activation = "softmax")(x)

VGG_model = keras.Model(VGG_model.input, x)
VGG_model.compile(loss = "categorical_crossentropy", optimizer = "adam", metrics = "accuracy")
VGG_model.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 48, 48, 3)]       0         
                                                                 
 block1_conv1 (Conv2D)       (None, 48, 48, 64)        1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 48, 48, 64)        36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 24, 24, 64)        0         
                                                                 
 block2_conv1 (Conv2D)       (None, 24, 24, 128)       73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 24, 24, 128)       147584    
                                                                 
 block2_pool (MaxPooling2D)  (None, 12, 12, 128)       0     

In [ ]:
hist = VGG_model.fit(training_set, validation_data = testing_set , epochs = 10)

In [ ]:
hist = hist.history

In [ ]:
#loss
plt.plot(hist['loss'], label='train loss')
plt.plot(hist['val_loss'],label='val loss')
plt.legend()
plt.show()
plt.savefig('E:\\Final_year Project\\VGG19_LossVal_loss')

#accuracy
plt.plot(hist['accuracy'], label='train acc')
plt.plot(hist['val_accuracy'],label='val acc')
plt.legend()
plt.show()
plt.savefig('E:\\Final_year Project\\VGG19_AccVal_acc')

In [ ]:
VGG_model.save("C:\\Users\\Lenovobhavya\\Documents\\Project_PBL\\data\\VGG19v1.h5")
model_json=VGG_model.to_json()
with open("C:\\Users\\Lenovobhavya\\Documents\\Project_PBL\\data\\VGG19.json", "w") as json_file:
    json_file.write(model_json)

In [ ]:
yhat_test = np.argmax(model.predict(X_test), axis=1)
ytest_ = np.argmax(y_test, axis=1)

scikitplot.metrics.plot_confusion_matrix(ytest_, yhat_test, figsize=(7,7))
pyplot.savefig("confusion_matrix_mobilenet.png")

test_accu = np.sum(ytest_ == yhat_test) / len(ytest_) * 100
print(f"test accuracy: {round(test_accu, 4)} %\n\n")

print(classification_report(ytest_, yhat_test))